<h1>1.3</h1>
In 1.3 we are required to compute the correlations between the gas consumption of every two households, and for each household, find out 5 other households with the highest correlation in gas consumption.

This part of the code reads the <b><i>hourly_readings_final</i></b>.csv file which is generated in the previous section (1.2) and outputs <b><i>top_correlated_households</i></b>.csv. The main problem faced in this section is how the correlation should be computed. The initial idea was to calculate only based on hourly readings computed in 1.2 without considering time. However, many of the households do not have complete data, which means in some month, these households do not have any meter readings. Thus simply computing correlation based on hourly readings is not sufficient.

The method used for 1.3 is to compute the average gas consumption for each household in each hour. The hourly consumption is computed first by comparing the readings between adjacent readings for the same household. After getting the hourly consumption, the mean of all readings in each hour is computed, as it is reasonable to say that by including more data points, the average can be more representative compared to taking a few data points. For each household, there are 24 data points for average hourly consumption, representing the 24 hours. Correlation matrix can be computed from the average hourly consumption, and the top 5 households with highest correlation can be found

In [4]:
import pandas as pd
import numpy as np
import datetime as dt 
from collections import namedtuple

In [ ]:
# 1.3
# import result obtained from 1.2
hourly_reading = pd.read_csv('./hourly_readings_final.csv')

# get the list of dataids
idList = hourly_reading['dataid'].drop_duplicates(keep = 'first')
idList = idList.reset_index(drop = True)

# split the column 'localminute' into 'day' and 'time'
new_time = hourly_reading['localminute'].str.split(" ", n = 1, expand = True)
hourly_reading['day'] = new_time[0]
hourly_reading['time'] = new_time[1]
hourly_reading.drop(columns = ['localminute'], inplace = True)

In [ ]:
# add the 'diffs' column which shows the hourly consumption by each household
# the function diff() computes the difference between the current data point and the previous data point
# therefore the data points at the boundary (data id changes) are wrong and will be changed to NaN
hourly_reading['diffs'] = hourly_reading['meter_value'].diff()
mask = hourly_reading.dataid != hourly_reading.dataid.shift(1)
hourly_reading['diffs'][mask] = np.nan

# compute the average hourly consumption by each household
# save all the values in the average_hourly_consumption dataframe created
HH_count = 0
average_hourly_consumption = pd.DataFrame(np.random.randint(low = 0.0, high = 10.0, size = (24, len(idList))), columns = idList)
for col in idList:
    print(col)
    for hr in range (0, 23):
#         set the criteria for selecting the data points
        if hr < 10:
            criterion = hourly_reading['time'].map(lambda x: (x[0] ==  '0') & (x[1] == str(hr)))
        if hr >= 10:
            criterion = hourly_reading['time'].map(lambda x: (x[0] == str(hr)[0]) & (x[1] ==  str(hr)[1]))
#         compute the mean of hourly consumption for each household and insert into average_hourly_consumption dataframe
        average_hourly_consumption[col].iloc[hr] = hourly_reading.loc[(hourly_reading.dataid == col) & criterion]['diffs'].mean()
    HH_count += 1

In [ ]:
# compute the correlation matrix of the new dataframe
corr = average_hourly_consumption.corr()

# remove the 1s in diagonal
# as the correlation between a column and itself is always 1
corr -= np.eye(corr.shape[0])

# create a new dataframe with random values inserted
top_corr = pd.DataFrame(np.random.randint(low = 0.0, high = 10.0, size = (5 * len(idList), 3)), columns = ['HH1', 'HH2', 'corr']).reset_index(drop = True)

# set 'corr' column data type as float64
# since correlation value is a floating point between 0 and 1
top_corr['corr'] = top_corr['corr'].astype(float)

In [ ]:
# loop through all data ids in the correlation matrix
# for each data id, find out the 5 data ids with the highest correlation values
# collate all results in the top_corr dataframe
count = 0
for i in corr.columns[:]:
    temp_corr = corr.nlargest(5, i)
    for j in range (0, 5):
        top_corr['HH1'].iloc[count] = i
        top_corr['HH2'].iloc[count] = temp_corr.index[j]
        top_corr['corr'].iloc[count] = temp_corr[i].iloc[j]
        count += 1

# export top_corr dataframe as csv file
top_corr.to_csv('top_correlated_households.csv', index = False)